<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figuren/book_cover_small.jpg"> Dieses Notebook ist Teil eines Begleit-Kurses zu dem im Springer-Verlag erschienenem Buch [Einführung in Unix/Linux für Naturwissenschaftler](http://www.springer.com/de/book/9783662503003)

Das Material wird Ihnen unter der [CC BY  4.0 Lizenz](http://creativecommons.org/licenses/by/4.0/deed.de) zur Verfügung gestellt. Wenn Ihnen der Kurs weiterhilft und Sie mich bei der Erstellung weiterer kostenloser Tutorials unterstützen möchten, überlegen Sie sich bitte, [das Buch zu kaufen](http://www.springer.com/de/book/9783662503003).

<!--NAVIGATION-->
< [Vorherige Lektion](04_Shell_Pipelines_und_Filter.ipynb) || [Verständnisfragen zu dieser Lektion](Verstaendnisfragen_zu_Lektion_05_for_Schleife.ipynb) || [Nächste Lektion](06_Shell_Shell-Skripte.ipynb) >

# Die `for`-Schleife

## Was behandeln wir in diesem Notebook / Lernziele

<div class="alert alert-success">

<b>Fragestellung:</b>
<ul>
    <li> Wie kann ich existierende <code>Unix</code>-Programme kombinieren, um neue und komplexere Aufgaben zu erledigen?</li>
</ul>

<b>Aspekte der Fragestellung:</b>
<ul>
    <li> Reproduzierbarkeit und Automatisierung von Aufgaben durch Schleifen</li>
</ul>

<b> Zeitaufwand für diese Lektion: </b> 
<ul>
    <li> Durcharbeiten des Textes: 30 min</li>
    <li> Verständnisfragen: 20 min</li>
</ul>
</div>

## Schleifen
Schleifen sind ein wichtiges Konstrukt der Programmierung. Sie erlauben es uns, Befehlssequenzen automatisch und reproduzierbar auf jedes Element *einer Liste* anzuwenden. Neben den in der letzten Lektion besprochenen Pipelines sind sie der zweite wichtige Baustein, um `Unix`-Kommandos für komplexe Aufgaben zu kombinieren. Sie sind essentiell, um langweilige und sich wiederholende Aufgaben fehlerfrei, effektiv und mit einem Mindestmaß an manueller Intervention durchzuführen.

## Die `for`-Schleife
Die `bash`-Shell bietet mehrere Schleifenkonstrukte und wir werden uns in diesem Tutorial auf die so genannte `for`-Schleife beschränken.

Annika arbeitet noch an ihrem Seminar zu den Monden der Planeten unseres Sonnensystems.

In [ ]:
cd ~/Seminar/Planeten_Daten
ls

Sie möchte für jeden Planeten den Mond mit der geringsten Umlaufperiode ermitteln. Mit der Struktur der Planetendateien (siehe [hier](04_Shell_Pipelines_und_Filter.ipynb#Planetendatei_Struktur)) ist dies der Mond mit dem niedrigsten Wert in der vierten Spalte. Um diesen für Jupiter zu erhalten, verwendet Annika die Pipeline

In [ ]:
# zeige den Mond mit der geringsten Umlaufperiode um Jupiter
sort -g -k 4 Jupiter.dat | head -n 1

Die Option `-k 4` für `sort` sorgt dafür, dass *`sort `die ersten drei Spalten nicht verwendet und erst der Inhalt ab Spalte vier für die Sortierung herangezogen wird*. Dies ist im vorliegenden Fall effektiv eine Sortierung nach der Umlaufzeit der Monde.

Um diese Aufgabe für jeden Planeten zu lösen, muss obige Pipeline jedes mal mit der entsprechenden Planetendatei neu ausgeführt werden. Das Problem, exakt dieselbe Befehlssequenz mit einer *Variablen* (hier die Planetendateien) wiederholt ausführen zu müssen, ist ein typischer Anwendungsfall für eine Schleife.

Die allgemeine Struktur der `for`-Schleife ist wie folgt:
```bash
for VARIABLE in LISTE
do
  BEFEHLSSEQUENZ mit ${VARIABLE}
done
```
Eine genauere Erklärung folgt weiter unten.

In unserem konkreten Fall sieht das, zunächst für die zwei Planeten Jupiter und Saturn, wie folgt aus

In [ ]:
# Schleife, um den Befehl sort -g -k 4 ..... | head -n 1
# auf die 'Liste' Jupiter.dat und Saturn.dat anzuwenden
#
for DATEI in Jupiter.dat Saturn.dat
do
  sort -g -k 4 ${DATEI} | head -n 1
done  

Das Schlüsselwort `for` sagt der Shell, dass eine Befehlssequenz für jedes Element einer *Liste* zu wiederholen ist. In jedem Schleifendurchlauf (auch Iteration genannt) wird ein Element der Liste einer *Schleifenvariable* zugeordnet und die Befehle innerhalb der Schleife werden ausgeführt, bevor zum nächsten Listenelement gesprungen wird. Innerhalb der Schleife können wir auf den *Wert* der *Schleifenvariable* mit dem Konstrukt `${...}` zugreifen, wobei `...` für den *Variablennamen* stehen.

Im konkreten Fall hat die Liste zwei Elemente, die Dateinamen `Jupiter.dat` und `Saturn.dat`. In jedem der zwei Schleifendurchläufe wird ein Dateiname der Schleifenvariable `DATEI` zugewiesen und der `sort`-Befehl ausgeführt. In der ersten Iteration hat die Variable `DATEI` den *Wert* `Jupiter.dat`, auf welchen mit `${DATEI}` zugegriffen wird. Es wird also in der ersten Iteration  der Befehl `sort -g -k 4 Jupiter.dat | head -n 1` ausgeführt. Das Ganze wiederholt sich in der zweiten Iteration, wobei der Wert der Variable `DATEI` jetzt `Saturn.dat` ist. Da die Liste im gegebenem Fall nur zwei Elemente hat, endet die Schleife nach zwei Iterationen.

Der allgemeine Programmfluss einer Schleife und die konkreten Schleifenelemente des aktuellen Beispiels sind noch einmal in folgender Figur dargestellt.

<img src="figuren/Shell_for_Schleife_fig1_und_2.png" style="width: 600px;">

Wir haben die Schleife bisher für Jupiter und Saturn verwendet. Um sie auf alle Planeten anzuwenden, müssen wir die erforderliche Liste *nicht* manuell erzeugen, sondern wir können sie einfach und effizient mit Wildcards von der Shell generieren lassen!

In [ ]:
# Wende die Schleife auf alle Dateien mit der Endung '.dat' an.
# Hier entspricht dies allen Planetendateien.
#
for DATEI in *.dat
do
  sort -g -k 4 ${DATEI} | head -n 1
done  

## Verschiedenes im Zusammenhang mit der  `for`-Schleife

### Hilfe bei der Schleifenkonstruktion

Anfängern fällt es oft schwer, Schleifen korrekt aufzubauen und damit verbundene Fehler sind nach Schleifendurchführung manchmal schwer zu finden. Eine Hilfe ist es, sich vor der aktuellen Schleifenausführung die Listenelemente und die Schleifenbefehle anzeigen zu lassen und sich so zu vergewissern, dass alles *korrekt* ist. Hierbei hilft das Kommando `echo`, welches unter anderem in Anführungsstrichen eingebetteten Text auf dem Bildschirm ausgeben kann.

In [ ]:
echo "Annika Oliver Thomas"  # gibt einfach 'Annika Oliver Thomas' aus

Interessant ist dieser Befehl in Zusammenhang mit Schleifen, da Variablen *vor der Ausgabe* ausgewertet werden. 

In [ ]:
# Die Schleife von oben. Der Schleifenbefehl ist aber in
# einen echo-Befehl eingebettet. Er wird dadurch angezeigt, aber
# nicht ausgeführt.
#
for DATEI in *.dat
do
  echo "sort -g -k 4 ${DATEI} | head -n 1"
done  

Mit diesem Trick kann man sich die Befehle, die die Schleife ausführen würde, erst einmal ansehen, *bevor* sie abgearbeitet werden. Wenn alles in Ordnung ist, entfernt man den `echo`-Befehl samt Anführungszeichen und führt die Schleife aus.

### Sukzessive Konstruktion einer Textdatei mit einer Schleife

Annika möchte sich die Monde mit den geringsten Umlaufzeiten nicht nur ausgeben lassen, sondern diese in eine Datei `schnelle_Monde.txt` speichern. Um dies zu erreichen, gibt es eine weitere Form der Ausgabeumlenkung, (siehe [die vierte Lektion](04_Shell_Pipelines_und_Filter.ipynb#Ausgabeumlenkung)). Die bisher betrachtete Ausgabeumlenkung leitet die Ausgabe eines Programms in eine Textdatei um. Bei Existenz dieser Datei wird diese allerdings *überschrieben*.

In [ ]:
echo "Annika" > test.txt   # lenke Annika in die Datei test.txt
cat test.txt

In [ ]:
echo "Oliver" > test.txt   # lenkt Oliver in Datei test.txt;
                           # Annika wird überschrieben.
cat test.txt                           

Es ist also mit dem `>`-Operator nicht möglich, eine Datei schrittweise aufzubauen und zu erweitern. Mit dem zu `>` verwandten Operator `>>` wird ebenfalls die Ausgabe eines Befehls in eine Datei umgelenkt. Falls die Ausgabedatei noch nicht existiert, so haben beide Operatoren denselben Effekt. Existiert die Datei jedoch, so wird die Ausgabe mit `>>` *an die bestehende Datei angehängt*! 

In [ ]:
echo "Annika" > test.txt    # lenke Annika in test.txt
cat test.txt

In [ ]:
echo "Oliver" >> test.txt   # hängt Oliver an test.txt an
cat test.txt                           

Hiermit kann Annika ihre Datei `schnelle_Monde.txt` erstellen.

In [ ]:
# Die schnellsten Monde werden alle in eine Datei
# geschrieben.
#
for DATEI in *.dat
do
  sort -g -k 4 ${DATEI} | head -n 1 >> schnelle_Monde.txt
done 
cat schnelle_Monde.txt

### Variablennamen

Wir haben in unserem Beispiel für den Variablennamen der Schleife `DATEI` gewählt, da es sich bei den Listenelementen um Dateinamen handelt. Es ist jedoch für die Funktionsweise der Schleife egal, welchen Namen wir wählen. So wäre unsere Schleife von eben äquivalent zu folgender Variante

In [ ]:
for x in *.dat
do
  sort -g -k 4 ${x} | head -n 1
done 

Aus Gründen der Lesbarkeit sollte jedoch ein dem Problem angepasster Name gewählt werden. Das hilft Ihnen und anderen, Ihre Schleifen besser und einfacher zu verstehen - vor allem, wenn Sie diese nach längerer Zeit wieder bearbeiten und an ein neues Problem anpassen müssen. Obwohl es nicht zwingend erforderlich ist, hat es sich auch eingebürgert, für Schleifenvariablen nur Großbuchstaben zu verwenden. Hiermit sind sie von anderen Schleifenelementen wie Kommandonamen oder Optionen sofort zu unterscheiden.

### Schleifen in der Shell

Die `for`-Schleife hat `drei` wesentliche Bestendteile: (1) Die Definition `for VARIABLE in LISTE`, was die Schleife einleitet und die Schleifenvariable und die Schleifenliste definiert; (2) Das Schlüsselwort `do`, welches den Schleifenbefehlsblock einleitet samt den eigentlichen Schleifenbefehlen und (3) das Schlüsselwort `done`, welches die Schleife abschließt. Hier in den Notebookzellen können wir diese Bestandteile schön formatiert darstellen, womit die Schleifenstruktur klar wird. Beachten Sie auch, dass wir die Schleifenbefehle etwas *einrücken*, um diesen Befehlsblock klar vom Rest hervorzuheben. Falls wir später neue Befehle in die Schleife einfügen müssen, wird sofort klar, wo dies zu tun ist. Betrachten wir das bisherige Beispiel, welches wir um einen zweiten Schleifenbefehl erweitern.

In [ ]:
# gut formatierte und übersichtliche Schleife
#
for DATEI in *.dat
do
  echo "Arbeite an Datei ${DATEI}"
  sort -g -k 4 ${DATEI} | head -n 1
done 

Innerhalb einer `Unix`-Terminalzeile ist es leider nicht ohne Weiteres möglich, eine Schleife so schön formatiert darzustellen. Hier muss alles *innerhalb einer Zeile* untergebracht werden. Dies wird durch Strichpunkte vor dem `do`, das den Befehlsblock einleitet, und nach jedem der Schleifenbefehle erreicht.

In [ ]:
# innerhalb einer einzigen Unix-Terminalzeile sähe obige
# Schleife wie folgt aus. Um alles in einer Zeile zu setzen,
# sind Strichpunkte for dem 'do' und nach jedem der
# Schleifenbefehle nötig.
#
for DATEI in *dat; do echo "Arbeite an Datei ${DATEI}"; sort -g -k 4 ${DATEI} | head -n 1; done

Wir erwähnen dies, damit Sie Schleifen in der Terminalzeilenstruktur gegebenenfalls verstehen und analysieren können. Wie Sie auch bei der Arbeit innerhalb eines `Unix`-Terminals ordentlich formatierte Schleifen erstellen und warten können, besprechen wir in der nächsten Lektion.
<a id='Statistik_Beobachtungen'></a>

## Annikas Bachelor-Arbeit: Statistiken aller Beobachtungen

Nachdem Annika bereits überprüft hat, wie vollständig ihre Beobachtungen sind (siehe [hier](04_Shell_Pipelines_und_Filter.ipynb#Annika_Test_Beobachtungen)), möchte sie jetzt die Qualität der vorhandenen Daten testen. 

In [ ]:
cd ~/Bachelor_Arbeit/Beobachtungen
ls

Sie hat von ihrem Betreuer das Programm `calc_stats.py` bekommen, welches den Mittelwert und die Standardabweichung einer Beobachtung berechnen kann. Annika testet zuerst, ob dieses Programm funktioniert und ob sie es richtig verwendet.

In [ ]:
# Berechne Mittwlwert und Standardabweichung einer Beobachtung
# im processed Unterverzeichnis
# Ausgabe des Programms ist:
#   Beobachtung   Mittelwert  Standardabweichung
python3 calc_stats.py processed/695833p.txt

Annika weiß, dass sie in jeder Beobachtung einen Mittelwert um 0 und eine Standardabweichung um 1 zu erwarten hat. Sie sieht daher diesen ersten Schritt als erledigt an. Als nächstes möchte sie das Programm mit einer Schleife über *alle* Beobachtungsdateien im Unterverzeichnis `processed` laufen lassen. Da Annika gerade anfängt, `for`-Schleifen zu programmieren und da die Analyse einige Zeit dauern wird, geht sie schrittweise vor. 

Als erstes möchte sie nur einen kleinen Teil ihrer Daten betrachten und wählt einen Testdatensatz aus.

In [ ]:
# Wähle einen Testdatensatz, um die Analyseschleife zu entwickeln und
# zu testen; kopiere diese Daten in ein separates test-Verzeichnis
mkdir test
cp ./processed/69583*p.txt ./test
ls test

Jetzt setzt sie eine erste Schleife über die Testdaten auf und überprüft, dass die Schleifenbefehle und die Schleifenliste korrekt sind.
<a id='for-statistik'></a>

In [ ]:
# Testschleife zur Verifizierung der Listenelemente
# und der Schleifenbefehle
for BEOBACHTUNG in test/*p.txt
do
  echo "python3 calc_stats.py ${BEOBACHTUNG}"
done

Hier sieht alles gut aus und Annika wendet die Analyseschleife auf die Testdaten an.

In [ ]:
# Analyseschleife über die Testdaten
#
for BEOBACHTUNG in test/*p.txt
do
  python3 calc_stats.py ${BEOBACHTUNG}
done

Nachdem auch dies erfolgreich ist, würde Annika als letzten Schritt die Analyseergebnisse der Schleife in eine Ergebnisdatei speichern.  

In [ ]:
# Analyseschleife über die Testdaten
# nur Speicherung der Ergebnisse
#
for BEOBACHTUNG in test/*p.txt
do
  python3 calc_stats.py ${BEOBACHTUNG} >> test_resultat.txt
done
cat test_resultat.txt

Nach diesen Vorarbeiten kann Annika ihre Schleife guten Gewissens auf den vollen Datensatz anwenden. Sie muss hierzu lediglich noch die Schleifenliste `test/*p.txt` durch `processed/*p.txt` ersetzen. Da dies ein wenig dauern würde, verzichten wir hier darauf.

<div class="alert alert-success">
<b>Zum Mitnehmen</b>
<ul>
    <li> <code>befehl >> datei</code> hängt die Ausgabe von <code>befehl</code> an <code>datei</code> an.</li>
    <li> <code>echo "text"</code> Gibt <code>text</code> auf dem Bildschirm aus.</li>
    <li> Die <code>for</code>-Schleife kann eine Befehlssequenz auf alle Elemente einer Liste ausführen.</li>
    <li> Wählen Sie für wissenschaftliche Daten konsistente Dateinamen, die einfach mit Wildcards anzusprechen sind. Dies macht es Ihnen einfach, sie effizient mit Schleifen zu verarbeiten.</li>
    <li> Entwickeln Sie komplizierte Schleifen schrittweise und testen Sie diese mit kleinen Datensätzen.</li>
</ul>
</div>

<!--NAVIGATION-->
< [Vorherige Lektion](04_Shell_Pipelines_und_Filter.ipynb) || [Verständnisfragen zu dieser Lektion](Verstaendnisfragen_zu_Lektion_05_for_Schleife.ipynb) || [Nächste Lektion](06_Shell_Shell-Skripte.ipynb) >